In [1]:
# stdlib
import os
from os import environ as env
from time import sleep

# syft absolute
import syft as sy
from syft.util.test_helpers.email_helpers import SENDER
from syft.util.test_helpers.email_helpers import create_user
from syft.util.test_helpers.email_helpers import get_email_server
from syft.util.test_helpers.email_helpers import make_user
from syft.util.test_helpers.email_helpers import save_users

In [2]:
ADMIN_EMAIL, ADMIN_PW = "admin@bigquery.org", "bqpw"

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

'python'

In [3]:
# in case we are not in k8s we set them here for orchestra to use
env["DEFAULT_ROOT_EMAIL"] = ADMIN_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ADMIN_PW

In [4]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

Autoreload enabled
Starting bigquery-high server on 0.0.0.0:8080
Found `reset=True` in the launch configuration. Resetting the server...


INFO:     Will watch for changes in these directories: ['/home/ionesio/workspace/PySyft/packages/syft/src/syft']
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)
INFO:     Started reloader process [13967] using WatchFiles


Waiting for server to start..

INFO:     Starting server with settings: {'name': 'bigquery-high', 'server_type': <ServerType.DATASITE: 'datasite'>, 'server_side_type': <ServerSideType.HIGH_SIDE: 'high'>, 'deployment_type': <DeploymentType.PYTHON: 'python'>, 'processes': 1, 'reset': False, 'dev_mode': True, 'enable_warnings': False, 'in_memory_workers': True, 'queue_port': None, 'create_producer': True, 'n_consumers': 1, 'association_request_auto_approval': False, 'background_tasks': False, 'db_config': None, 'db_url': None} and worker class: <class 'syft.server.datasite.Datasite'>


.

INFO:     Started server process [13981]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


. Done.


SyftInfo: You have launched a development server at http://0.0.0.0:8080. It is intended only for local use.

INFO:     Stopping reloader process [13967]


In [5]:
email_server, smtp_server = get_email_server()

> Starting SMTPTestServer on: 0.0.0.0:9025
> Starting SMTPTestServer on: 0.0.0.0:9025


In [6]:
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

Logged into <bigquery-high: High side Datasite> as <admin@bigquery.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

In [7]:
high_client.api.services.settings.enable_notifications(
    email_sender=SENDER,
    email_server="localhost",
    email_port="9025",
)

> SMTPTestServer got an email for ['admin@bigquery.org']
> SMTPTestServer got an email for ['admin@bigquery.org']


# Check default email instant deliver

In [8]:
@sy.syft_function_single_use()
def test_syft_function():  # noqa: F811
    print("Testing syft function request email notification")
    return True


# Create a project
project = sy.Project(
    name="My Cool Project",
    description="""Hi, I want to calculate the mean of your private data,\
                    pretty please!""",
    members=[high_client],
)
project.create_code_request(test_syft_function, high_client)

SyftSuccess: Syft function 'test_syft_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Code request for 'test_syft_function' successfully added to 'My Cool Project' Project. To see code requests by a client, run `[your_client].code`

In [9]:
assert "A new request has been submitted and requires your attention." in email_server.load_emails()[ADMIN_EMAIL][-1].email_content

## Batch Notifications

In [10]:
current_admin_email_cache_len = len(email_server.load_emails()[ADMIN_EMAIL])

In [11]:
from syft.service.notifier.notifier_enums import EMAIL_TYPES, NOTIFICATION_FREQUENCY
high_client.settings.batch_notifications(EMAIL_TYPES.REQUEST_EMAIL, NOTIFICATION_FREQUENCY.DAILY, start_time="6:00 pm")

In [12]:
@sy.syft_function_single_use()
def test_syft_function_data_upload():  # noqa: F811
    print("Testing data upload request notification email")
    return True

project.create_code_request(test_syft_function_data_upload, high_client)

@sy.syft_function_single_use()
def test_syft_function_model_training():  # noqa: F811
    print("Testing model training request notification email")
    return True

project.create_code_request(test_syft_function_model_training, high_client)

@sy.syft_function_single_use()
def test_syft_function_data_analysis():  # noqa: F811
    print("Testing data analysis request notification email")
    return True

project.create_code_request(test_syft_function_data_analysis, high_client)

@sy.syft_function_single_use()
def test_syft_function_data_cleaning():  # noqa: F811
    print("Testing data cleaning request notification email")
    return True

project.create_code_request(test_syft_function_data_cleaning, high_client)

@sy.syft_function_single_use()
def test_syft_function_report_generation():  # noqa: F811
    print("Testing report generation request notification email")
    return True

project.create_code_request(test_syft_function_report_generation, high_client)

SyftSuccess: Syft function 'test_syft_function_data_upload' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'test_syft_function_model_training' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'test_syft_function_data_analysis' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'test_syft_function_data_cleaning' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Syft function 'test_syft_function_report_generation' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: Code request for 'test_syft_function_report_generation' successfully added to 'My Cool Project' Project. To see code requests by a client, run `[your_client].code`

In [13]:
assert current_admin_email_cache_len ==  len(email_server.load_emails()[ADMIN_EMAIL])

In [14]:
high_client.settings.batch_notifications(EMAIL_TYPES.REQUEST_EMAIL, NOTIFICATION_FREQUENCY.INSTANT)

In [ ]:
sleep(15)

In [15]:
assert current_admin_email_cache_len + 1 ==  len(email_server.load_emails()[ADMIN_EMAIL])

In [16]:
assert "Batched Requests Notification" in email_server.load_emails()[ADMIN_EMAIL][-1].email_content

## Test Setting batch notifications without passing start time

In [17]:
with sy.raises(sy.SyftException, show=True):
    high_client.settings.batch_notifications(EMAIL_TYPES.REQUEST_EMAIL, NOTIFICATION_FREQUENCY.DAILY)

with sy.raises successfully caught the following exception:


## Test setting batch notifications with bad formatted start time

In [19]:
with sy.raises(sy.SyftException, show=True):
    high_client.settings.batch_notifications(EMAIL_TYPES.REQUEST_EMAIL, NOTIFICATION_FREQUENCY.DAILY, "6 pm")

with sy.raises successfully caught the following exception:


In [20]:
smtp_server.stop()

> Stopping SMTPTestServer


In [21]:
server.land()

Stopping bigquery-high


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [13981]
